In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [18]:
df = pd.read_table('./data/germancredit.txt', header=None, sep=' ')
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [19]:
print(df.describe())

                1             4            7            10           12  \
count  1000.000000   1000.000000  1000.000000  1000.000000  1000.000000   
mean     20.903000   3271.258000     2.973000     2.845000    35.546000   
std      12.058814   2822.736876     1.118715     1.103718    11.375469   
min       4.000000    250.000000     1.000000     1.000000    19.000000   
25%      12.000000   1365.500000     2.000000     2.000000    27.000000   
50%      18.000000   2319.500000     3.000000     3.000000    33.000000   
75%      24.000000   3972.250000     4.000000     4.000000    42.000000   
max      72.000000  18424.000000     4.000000     4.000000    75.000000   

                15           17           20  
count  1000.000000  1000.000000  1000.000000  
mean      1.407000     1.155000     1.300000  
std       0.577654     0.362086     0.458487  
min       1.000000     1.000000     1.000000  
25%       1.000000     1.000000     1.000000  
50%       1.000000     1.000000     1.0000

In [22]:
# convert table to pd dataframe
df = pd.DataFrame(df)

# convert current string variables to categorical values
categoricals = [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]

for col in categoricals:
    df.iloc[:, col] = df.iloc[:, col].astype('category')

print(df.dtypes)

0     object
1      int64
2     object
3     object
4      int64
5     object
6     object
7      int64
8     object
9     object
10     int64
11    object
12     int64
13    object
14    object
15     int64
16    object
17     int64
18    object
19    object
20     int64
dtype: object


In [25]:
print(df.describe(include='all'))

          0            1     2     3             4     5     6            7   \
count   1000  1000.000000  1000  1000   1000.000000  1000  1000  1000.000000   
unique     4          NaN     5    10           NaN     5     5          NaN   
top      A14          NaN   A32   A43           NaN   A61   A73          NaN   
freq     394          NaN   530   280           NaN   603   339          NaN   
mean     NaN    20.903000   NaN   NaN   3271.258000   NaN   NaN     2.973000   
std      NaN    12.058814   NaN   NaN   2822.736876   NaN   NaN     1.118715   
min      NaN     4.000000   NaN   NaN    250.000000   NaN   NaN     1.000000   
25%      NaN    12.000000   NaN   NaN   1365.500000   NaN   NaN     2.000000   
50%      NaN    18.000000   NaN   NaN   2319.500000   NaN   NaN     3.000000   
75%      NaN    24.000000   NaN   NaN   3972.250000   NaN   NaN     4.000000   
max      NaN    72.000000   NaN   NaN  18424.000000   NaN   NaN     4.000000   

          8     9   ...    11          